In [23]:
import h5py as h5
import glob
import numpy as np
from datetime import datetime, timedelta

## I/O file

In [24]:
#output_dir = "/Users/song884/remote/reach/Outputs/2007_age/"
# output_dir = "/Users/shua784/Paraview/HFR/test_2007_age/"

# output_h5.close()
# output_file = output_dir + "all_age_flux.h5"

case_name = "HFR_model_100x100x2_cyclic/"

model_dir = "/global/cscratch1/sd/pshuai/" + case_name

fname_pflotran_h5 = model_dir + "TH_100x100x2_flow_age.h5"

fname_output_h5 = model_dir + "pflotran_100x100x2_cyclic_combo.h5"

fname_h5_old = model_dir + "backup/pflotran_100x100x2_cyclic.h5"
fname_h5_new = model_dir + "pflotran_100x100x2_cyclic.h5"

fname_h5_age = "/global/cscratch1/sd/pshuai/TH-100x100x2_steady_state/TH_100x100x2_steady_stage-restart.h5"
fname_h5_thermal = "/global/cscratch1/sd/pshuai/TH-100x100x2-thermal_new_origin/TH_100x100x2_flow_age-restart.h5"

In [4]:
# view the tree structure of hdf5 file
def print_structure(name, obj):
    print(name)
    
## example: input_h5.visititems(print_structure)

In [5]:
def batch_delta_to_time(origin, x, time_format, delta_format):
    y = []
    for ix in x:
        if delta_format == "hours":
            temp_y = origin + timedelta(hours=ix)
        elif delta_format == "days":
            temp_y = origin + timedelta(days=ix)
        elif delta_format == "minutes":
            temp_y = origin + timedelta(minutes=ix)
        elif delta_format == "weeks":
            temp_y = origin + timedelta(weeks=ix)
        elif delta_format == "seconds":
            temp_y = origin + timedelta(seconds=ix)
        elif delta_format == "microseconds":
            temp_y = origin + timedelta(microseconds=ix)
        elif delta_format == "milliseconds":
            temp_y = origin + timedelta(milliseconds=ix)
        else:
            print("Sorry, this naive program only solve single time unit")
        y.append(temp_y.strftime(time_format))
    y = np.asarray(y)
    return(y)

In [6]:
date_origin = datetime.strptime("2007-03-28 12:00:00", "%Y-%m-%d %H:%M:%S")

# Combine multiple hdf5 files

In [46]:
output_h5 = h5.File(fname_output_h5, "w")

In [47]:
# all_h5 = glob.glob(output_dir + "pflotran*h5")
# all_h5 = np.sort(all_h5)

**copy the 1st h5 file's groups (`Coordinates`, `Provenance`, `Time: xxx h` ...)**

In [48]:
# i_h5 = all_h5[0]
# print(i_h5)
input_h5 = h5.File(fname_h5_old, "r")

groups = list(input_h5.keys())
for i_group in groups:
    print(i_group)
    group_id = output_h5.require_group(i_group)
    datasets = list(input_h5[i_group].keys())
    for i_dataset in datasets:
        input_h5.copy("/" + i_group + "/" + i_dataset,
                      group_id, name=i_dataset)
input_h5.close()

Coordinates
Provenance
Time:  3.29880E+04 h
Time:  3.31080E+04 h
Time:  3.32280E+04 h
Time:  3.33480E+04 h
Time:  3.34680E+04 h
Time:  3.35880E+04 h
Time:  3.37080E+04 h
Time:  3.38280E+04 h
Time:  3.39480E+04 h
Time:  3.40680E+04 h
Time:  3.41880E+04 h
Time:  3.43080E+04 h
Time:  3.44280E+04 h
Time:  3.45480E+04 h
Time:  3.46680E+04 h
Time:  3.47880E+04 h
Time:  3.49080E+04 h
Time:  3.50280E+04 h
Time:  3.51480E+04 h
Time:  3.52680E+04 h
Time:  3.53880E+04 h
Time:  3.55080E+04 h
Time:  3.56280E+04 h
Time:  3.57480E+04 h
Time:  3.58680E+04 h
Time:  3.59880E+04 h
Time:  3.61080E+04 h
Time:  3.62280E+04 h
Time:  3.63480E+04 h
Time:  3.64680E+04 h
Time:  3.65880E+04 h
Time:  3.67080E+04 h
Time:  3.68280E+04 h
Time:  3.69480E+04 h
Time:  3.70680E+04 h
Time:  3.71880E+04 h
Time:  3.73080E+04 h
Time:  3.74280E+04 h
Time:  3.75480E+04 h
Time:  3.76680E+04 h
Time:  3.77880E+04 h
Time:  3.79080E+04 h
Time:  3.80280E+04 h
Time:  3.81480E+04 h
Time:  3.82680E+04 h
Time:  3.83880E+04 h
Time:  3.85

**copy the 2nd h5 file's group with `Time: xxx h`** 

In [49]:
# for i_h5 in all_h5[1:]:
#     print(i_h5)
input_h5 = h5.File(fname_h5_new, "r")

groups = list(input_h5.keys())
groups = [s for s,  s in enumerate(groups) if "Time:" in s]
for i_group in groups:
    print(i_group)
    group_id = output_h5.require_group(i_group)
    datasets = list(input_h5[i_group].keys())
    for i_dataset in datasets:
        input_h5.copy("/" + i_group + "/" + i_dataset,
                      group_id, name=i_dataset)
input_h5.close()

Time:  6.17880E+04 h
Time:  6.19080E+04 h
Time:  6.20280E+04 h
Time:  6.21480E+04 h
Time:  6.22680E+04 h
Time:  6.23880E+04 h
Time:  6.25080E+04 h
Time:  6.26280E+04 h
Time:  6.27480E+04 h
Time:  6.28680E+04 h
Time:  6.29880E+04 h
Time:  6.31080E+04 h
Time:  6.32280E+04 h
Time:  6.33480E+04 h
Time:  6.34680E+04 h
Time:  6.35880E+04 h
Time:  6.37080E+04 h
Time:  6.38280E+04 h
Time:  6.39480E+04 h
Time:  6.40680E+04 h
Time:  6.41880E+04 h
Time:  6.43080E+04 h
Time:  6.44280E+04 h
Time:  6.45480E+04 h
Time:  6.46680E+04 h
Time:  6.47880E+04 h
Time:  6.49080E+04 h
Time:  6.50280E+04 h
Time:  6.51480E+04 h
Time:  6.52680E+04 h
Time:  6.53880E+04 h
Time:  6.55080E+04 h
Time:  6.56280E+04 h
Time:  6.57480E+04 h
Time:  6.58680E+04 h
Time:  6.59880E+04 h
Time:  6.61080E+04 h
Time:  6.62280E+04 h
Time:  6.63480E+04 h
Time:  6.64680E+04 h
Time:  6.65880E+04 h
Time:  6.67080E+04 h
Time:  6.68280E+04 h
Time:  6.69480E+04 h
Time:  6.70680E+04 h
Time:  6.71880E+04 h
Time:  6.73080E+04 h
Time:  6.7428

In [50]:
output_h5.close()

# Extract output from a single hdf5 file

In [20]:
input_h5 = h5.File(fname_pflotran_h5, "r")

output_h5 = h5.File(fname_output_h5, "w")

In [8]:
input_h5.visititems(print_structure)

Coordinates
Coordinates/X [m]
Coordinates/Y [m]
Coordinates/Z [m]
Provenance
Provenance/PETSc
Provenance/PETSc/detail_petsc_config
Provenance/PETSc/detail_petsc_parent
Provenance/PETSc/detail_petsc_status
Provenance/PFLOTRAN
Provenance/PFLOTRAN/detail_pflotran_diff
Provenance/PFLOTRAN/detail_pflotran_fflags
Provenance/PFLOTRAN/detail_pflotran_parent
Provenance/PFLOTRAN/detail_pflotran_status
Provenance/PFLOTRAN/pflotran_compile_date_time
Provenance/PFLOTRAN/pflotran_compile_hostname
Provenance/PFLOTRAN/pflotran_compile_user
Provenance/PFLOTRAN/pflotran_input_file
Time:  3.24000E+04 h
Time:  3.24000E+04 h/Kludged_material_ids_for_VisIt
Time:  3.24000E+04 h/Liquid X-Flux Velocities
Time:  3.24000E+04 h/Liquid X-Velocity [m_per_h]
Time:  3.24000E+04 h/Liquid Y-Flux Velocities
Time:  3.24000E+04 h/Liquid Y-Velocity [m_per_h]
Time:  3.24000E+04 h/Liquid Z-Flux Velocities
Time:  3.24000E+04 h/Liquid Z-Velocity [m_per_h]
Time:  3.24000E+04 h/Liquid_Head [m]
Time:  3.24000E+04 h/Liquid_Pressur

KeyboardInterrupt: 

In [9]:
groups = list(input_h5.keys())

ts_groups = [s for s,  s in enumerate(groups) if "Time:" in s]

In [10]:
# sort time based on scientific value
ts_groups = sorted(ts_groups, key = lambda time: float(time[7:18]))

In [11]:
real_time = [str(batch_delta_to_time(date_origin, [float(itime[7:18])], "%Y-%m-%d %H:%M:%S", "hours")[0])
              for itime in ts_groups]

**copy groups and dsets from `input_h5`**

**extract every 120h**

In [21]:
# copy groups before "Time: xxxx h"
for i_group in groups[:2]:
    print(i_group)
    group_id = output_h5.require_group(i_group)
    datasets = list(input_h5[i_group].keys())
    for i_dataset in datasets:
        input_h5.copy("/" + i_group + "/" + i_dataset,
                      group_id, name=i_dataset)

# copy each group "Time: xxxx h"
for i_group in ts_groups[98::20]:
    print(i_group)
    group_id = output_h5.require_group(i_group)
    datasets = list(input_h5[i_group].keys())
    
    # select to copy only subset datasets[::2] or the full dataset[::]
    for i_dataset in datasets[::2]:
        input_h5.copy("/" + i_group + "/" + i_dataset,
                      group_id, name=i_dataset)

Coordinates
Provenance
Time:  3.29880E+04 h
Time:  3.31080E+04 h
Time:  3.32280E+04 h
Time:  3.33480E+04 h
Time:  3.34680E+04 h
Time:  3.35880E+04 h
Time:  3.37080E+04 h
Time:  3.38280E+04 h
Time:  3.39480E+04 h
Time:  3.40680E+04 h
Time:  3.41880E+04 h
Time:  3.43080E+04 h
Time:  3.44280E+04 h
Time:  3.45480E+04 h
Time:  3.46680E+04 h
Time:  3.47880E+04 h
Time:  3.49080E+04 h
Time:  3.50280E+04 h
Time:  3.51480E+04 h
Time:  3.52680E+04 h
Time:  3.53880E+04 h
Time:  3.55080E+04 h
Time:  3.56280E+04 h
Time:  3.57480E+04 h
Time:  3.58680E+04 h
Time:  3.59880E+04 h
Time:  3.61080E+04 h
Time:  3.62280E+04 h
Time:  3.63480E+04 h
Time:  3.64680E+04 h
Time:  3.65880E+04 h
Time:  3.67080E+04 h
Time:  3.68280E+04 h
Time:  3.69480E+04 h
Time:  3.70680E+04 h
Time:  3.71880E+04 h
Time:  3.73080E+04 h
Time:  3.74280E+04 h
Time:  3.75480E+04 h
Time:  3.76680E+04 h
Time:  3.77880E+04 h
Time:  3.79080E+04 h
Time:  3.80280E+04 h
Time:  3.81480E+04 h
Time:  3.82680E+04 h
Time:  3.83880E+04 h
Time:  3.85

In [22]:
output_h5.close()

input_h5.close()

# rewrite part of HDF5 file

In [56]:
output_h5 = h5.File(fname_output_h5, "w")

input_h5_1 = h5.File(fname_h5_thermal, "r")
input_h5_2 = h5.File(fname_h5_age, "r")

**copy every dataset in the restart file of TH mode**

In [57]:
groups = list(input_h5_1.keys())
for i_group in groups:
    print(i_group)
    group_id = output_h5.require_group(i_group)
    datasets = list(input_h5_1[i_group].keys())
    for i_dataset in datasets:
        print(i_dataset)
        input_h5_1.copy("/" + i_group + "/" + i_dataset,
                      group_id, name=i_dataset)

Checkpoint
PMCSubsurfaceFlow
PMCSubsurfaceTransport
Revision Number


**get `Total_tracer` and `Age_tracer` from 100-yr age simulation, i.e. the `Primary_variable` under transport group**

In [59]:
grp_2 = input_h5_2['Checkpoint/PMCSubsurfaceTransport/transport']
dset_2 = grp_2['Primary_Variable']
list(grp_2.keys())

['Checkpoint_Activity_Coefs', 'NDOF', 'Primary_Variable']

**replace the original `Primary_Variable` with the new `Primary_Variable` from the age simulation**

In [61]:
grp_1 = output_h5['Checkpoint/PMCSubsurfaceTransport/transport']
del grp_1['Primary_Variable']
grp_1.create_dataset("Primary_Variable", data=dset_2)

<HDF5 dataset "Primary_Variable": shape (36000000,), type "<f8">

In [62]:
input_h5_1.close()
input_h5_2.close()
output_h5.close()